In [286]:
import pandas as pd
import openpyxl
from setup_data import (tasas, via_importacion, tipo_vhl, seg_casco,
                         seg_rc, seg_ovp, tasa_rc_min, tasa_rc_max, tasa_ovp_min, 
                         tasa_ovp_max, recargo_ptf, products_data)

In [291]:
file_path = "AUTO.xlsx"

main_df = pd.read_excel(file_path)

TypeError: <class 'openpyxl.styles.named_styles._NamedCellStyle'>.name should be <class 'str'> but value is <class 'NoneType'>

In [ ]:
df = main_df

In [ ]:
def updated_name_column(df):
    df['Updated Nombre Producto'] = df['Nombre Producto'].map(products_data)
    return df

In [ ]:
def add_cobertura_basica_column(df):
    df["Cobertura Básica / Casco"] = df["Auto Cober. Básica 1"] + df["Auto Cober. Básica 2"] + df["Auto Accesorios"]
    return df

In [ ]:
def round_to_million(n):
    if n > 150000000:
        return 150000000
    else:
        return (n // 1000000) * 1000000

In [ ]:
def add_capital_seg_column(df):
    df["Capital seg"] = df["Cobertura Básica / Casco"].apply(round_to_million)
    return df

In [ ]:
def add_resp_civil_column(df):
    df["Resp. Civil"] = df["Auto Lesión 2 o más Per."] + df["Auto Daños Material a Ter."]
    return df

In [ ]:
def add_ovp_column(df):
    df["OVP"] = (df["Auto Muerte/Incap."] + df["Auto Asist. Médica"]) *  df["Cant. Ocupantes"]
    return df

In [ ]:
def add_franquicia_column(df):
    def map_franquicia(value):
        if value < 500000:
            return "General"
        elif value == 500000:
            return "Franquicia 500mil"
        else:
            return "Franquicia 1 millón"

    df["Franquicia"] = df["Franquicia Monto Fijo"].apply(map_franquicia)
    return df

In [ ]:
def add_importacion_column(df):
    df["Importacion"] = df["Via Importación"].map(via_importacion)
    return df

In [ ]:
def add_tipo_column(df):
    df['Tipo'] = df['Tipo Vehiculo'].map(tipo_vhl)
    return df

In [ ]:
def add_segmento_column(df):
    df['Segmento'] = df['Tipo'] + df['Franquicia'] + df['Importacion']
    return df

In [ ]:
def add_segmento_casco_column(df):
    df['Segmento Casco'] = df['Segmento'].map(seg_casco)
    return df

def add_segmento_rc_column(df):
    df['Segmento RC'] = df['Segmento'].map(seg_rc)
    return df

def add_segmento_ovp_column(df):
    df['Segmento OVP'] = df['Segmento'].map(seg_ovp)
    return df

In [ ]:
def add_final_casco_column(df):
    df['Final Casco'] = df['Segmento Casco'] + "-" + df['Capital seg'].astype(str) 
    return df

In [ ]:
def add_tasa_min_prima_pura_casco_column(df):
    df['Tasa MIN Prima Pura Casco'] = df['Final Casco'].map(lambda x: tasas.get(x, {}).get('min'))
    return df 

def add_tasa_min_prima_pura_rc_column(df):
    df['Tasa MIN Prima Pura RC'] = df['Segmento RC'].map(tasa_rc_min)
    return df

def add_tasa_min_prima_pura_ovp_column(df ):
    df['Tasa MIN Prima Pura OVP'] = df['Segmento OVP'].map(tasa_ovp_min)
    return df

In [ ]:
def add_prima_min_cobertura_basica_casco_column(df):
    df['Prima MIN Cobertura Basica / Casco'] = df['Cobertura Básica / Casco'] * df['Tasa MIN Prima Pura Casco']
    return df
    
def add_prima_min_rc_column(df):
    df['Prima MIN RC'] = df["Resp. Civil"] * df["Tasa MIN Prima Pura RC"]
    return df

def add_prima_min_ovp_column(df):
    df['Prima MIN OVP'] = df["OVP"] * df["Tasa MIN Prima Pura OVP"]
    return df

In [ ]:
def add_dias_vigencia_column(df):
    df['Dias Vigencia'] = (df['Fec. Hasta'] - df['Fec. Desde']).dt.days + 1
    return df

def add_prima_tecnica_art_column(df):
    df['Prima Tecnica Art.'] = df['Prima Técnica Art.'] / df['Dias Vigencia'] * 365
    return df

In [ ]:
def add_min_prima_pura_cot_column(df):
    df['MIN Prima Pura Cot'] = df['Prima MIN Cobertura Basica / Casco'] + df['Prima MIN RC'] + df['Prima MIN OVP']
    return df

def add_min_prima_tarifa_cot_column(df):
    df['MIN Prima Tarifa Cot'] = df['MIN Prima Pura Cot'] * recargo_ptf
    return df

In [ ]:
def add_tasa_max_prima_pura_casco_column(df):
    df['Tasa MAX Prima Pura Casco'] = df['Final Casco'].map(lambda x: tasas.get(x, {}).get('max'))
    return df 

def add_tasa_max_prima_pura_rc_column(df):
    df['Tasa MAX Prima Pura RC'] = df['Segmento RC'].map(tasa_rc_max)
    return df

def add_tasa_max_prima_pura_ovp_column(df ):
    df['Tasa MAX Prima Pura OVP'] = df['Segmento OVP'].map(tasa_ovp_max)
    return df

In [ ]:
def add_prima_max_cobertura_basica_casco_column(df):
    df['Prima MAX Cobertura Basica / Casco'] = df['Cobertura Básica / Casco'] * df['Tasa MAX Prima Pura Casco']
    return df
    
def add_prima_max_rc_column(df):
    df['Prima MAX RC'] = df["Resp. Civil"] * df["Tasa MAX Prima Pura RC"]
    return df

def add_prima_max_ovp_column(df):
    df['Prima MAX OVP'] = df["OVP"] * df["Tasa MAX Prima Pura OVP"]
    return df

In [ ]:
def add_max_prima_pura_cot_column(df):
    df['MAX Prima Pura Cot'] = df['Prima MAX Cobertura Basica / Casco'] + df['Prima MAX RC'] + df['Prima MAX OVP']
    return df

def add_max_prima_tarifa_cot_column(df):
    df['MAX Prima Tarifa Cot'] = df['MAX Prima Pura Cot'] * recargo_ptf
    return df

In [ ]:
def add_monto_diferencia_prima_pura_column(df):
    def calculate_difference(row):
        cx = row['Prima Tecnica Art.']
        da = row['MIN Prima Pura Cot']
        dj = row['MAX Prima Tarifa Cot']

        if cx < da:
            return cx - da
        elif cx > dj:
            return cx - dj
        else:
            return ""

    df['Monto Diferencia Prima Pura'] = df.apply(calculate_difference, axis=1)
    return df

def add_monto_diferencia_prima_tarifa_column(df):
    def calculate_difference(row):
        cx = row['Prima Tecnica Art.']
        da = row['MIN Prima Tarifa Cot']
        dj = row['MAX Prima Tarifa Cot']

        if cx < da:
            return cx - da
        elif cx > dj:
            return cx - dj
        else:
            return ""

    df['Monto Diferencia Prima Tarifa'] = df.apply(calculate_difference, axis=1)
    return df

In [ ]:
print(df.columns)

Index(['Emitido', 'Sección Principal', 'Nombre Sección Principal', 'Sección',
       'Nombre Sección', 'Póliza', 'Endoso', 'Tipo Póliza',
       'Nombre Tipo Póliza', 'Contratante/Asegurado',
       ...
       'Stro. Auto Cobertura Básica 2', 'Stro. Auto Lesión Una Per. Art.',
       'Stro. Auto Lesión 2 o más Per. Art.', 'Stro. Auto Robo Veh. Art.',
       'Stro. Auto Accesorios Art.', 'Stro. Auto Daños Material a Ter. Art.',
       'Stro. Auto Muerte/Incap. Art.', 'Stro. Auto Asist. Médica Art.',
       'Stro. Auto Otras Coberturas Art.', 'Cantidad de Personas Articulo'],
      dtype='object', length=177)


In [ ]:
df = updated_name_column(df)
df = add_cobertura_basica_column(df)
df = add_capital_seg_column(df)
df = add_resp_civil_column(df)
df = add_ovp_column(df)
df = add_tipo_column(df)
df = add_franquicia_column(df)
df = add_importacion_column(df)
df = add_segmento_column(df)
df = add_segmento_casco_column(df)
df = add_segmento_rc_column(df)
df = add_segmento_ovp_column(df)
df = add_final_casco_column(df)
df = add_tasa_min_prima_pura_casco_column(df)
df = add_tasa_min_prima_pura_rc_column(df)
df = add_tasa_min_prima_pura_ovp_column(df)
df = add_prima_min_cobertura_basica_casco_column(df)
df = add_prima_min_rc_column(df)
df = add_prima_min_ovp_column(df)
df = add_dias_vigencia_column(df)
df = add_prima_tecnica_art_column(df)
df = add_min_prima_pura_cot_column(df)
df = add_min_prima_tarifa_cot_column(df)
df = add_tasa_max_prima_pura_casco_column(df)
df = add_tasa_max_prima_pura_rc_column(df)
df = add_tasa_max_prima_pura_ovp_column(df)
df = add_prima_max_cobertura_basica_casco_column(df)
df = add_prima_max_rc_column(df)
df = add_prima_max_ovp_column(df)
df = add_max_prima_pura_cot_column(df)
df = add_max_prima_tarifa_cot_column(df)
df = add_monto_diferencia_prima_pura_column(df)
df = add_monto_diferencia_prima_tarifa_column(df)
print(df)

     Emitido  Sección Principal Nombre Sección Principal  Sección  \
0          A                500              AUTOMOVILES      501   
1          A                500              AUTOMOVILES      501   
2          A                500              AUTOMOVILES      501   
3          A                500              AUTOMOVILES      501   
4          A                500              AUTOMOVILES      501   
...      ...                ...                      ...      ...   
6279       N                500              AUTOMOVILES      501   
6280       N                500              AUTOMOVILES      501   
6281       N                500              AUTOMOVILES      501   
6282       N                500              AUTOMOVILES      501   
6283       N                500              AUTOMOVILES      501   

     Nombre Sección  Póliza  Endoso  Tipo Póliza Nombre Tipo Póliza  \
0       AUTOMOVILES     551       0            2         Renovacion   
1       AUTOMOVILES     552  

In [ ]:
df.to_excel('output_updated_products.xlsx', index=False)

In [ ]:
def count_product_names_x(df):
    df['Mapped Nombre Producto'] = df['Nombre Producto'].map(products_data)
    df_count = df.groupby('Updated Nombre Producto').agg({
        'Mapped Nombre Producto': 'count',
        'Prima Tecnica Art.': 'sum',
        'MIN Prima Pura Cot': 'sum',
        'MIN Prima Tarifa Cot': 'sum',
        'MAX Prima Pura Cot': 'sum'
    }).reset_index()
    df_count.columns = ['PRODUCTO', 'CANTIDAD', 'PRIMA TECNICA ART', 'MIN PRIMA PURA COT', 'MIN PRIMA TARIFA COT', 'MAX PRIMA PURA COT']
    return df_count

In [ ]:
def count_product_names_x2(df):
    # Convert columns to numeric type and round to 2 decimal places
    numeric_columns = ['Prima Tecnica Art.', 'MIN Prima Pura Cot', 'MIN Prima Tarifa Cot', 'MAX Prima Pura Cot', 'Monto Diferencia Prima Pura', 'Monto Diferencia Prima Tarifa']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').round(2)

    df['Mapped Nombre Producto'] = df['Nombre Producto'].map(products_data)
    df_grouped = df.groupby('Updated Nombre Producto').agg({
        'Mapped Nombre Producto': 'count',
        'Prima Tecnica Art.': 'sum',
        'MIN Prima Pura Cot': 'sum',
        'MIN Prima Tarifa Cot': 'sum',
        'MAX Prima Pura Cot': 'sum',
        'Monto Diferencia Prima Pura': 'sum',
        'Monto Diferencia Prima Tarifa': 'sum'
    }).reset_index()

    df_grouped.columns = ['PRODUCTO', 'CANTIDAD', 'PRIMA TECNICA ART', 'MIN PRIMA PURA COT', 'MIN PRIMA TARIFA COT', 'MAX PRIMA PURA COT', 'Total Monto Diferencia Prima Pura', 'Total Monto Diferencia Prima Tarifa']
    return df_grouped

In [ ]:
def process_resumen(df):
    # Convert columns to numeric type and round to 2 decimal places
    numeric_columns = ['Prima Tecnica Art.', 'MIN Prima Pura Cot', 'MIN Prima Tarifa Cot', 'MAX Prima Pura Cot', 'Monto Diferencia Prima Pura', 'Monto Diferencia Prima Tarifa']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').round(2)

    df['Mapped Nombre Producto'] = df['Nombre Producto'].map(products_data)
    df_grouped = df.groupby('Updated Nombre Producto').agg({
        'Mapped Nombre Producto': 'count',
        'Prima Tecnica Art.': 'sum',
        'MIN Prima Pura Cot': 'sum',
        'MIN Prima Tarifa Cot': 'sum',
        'MAX Prima Pura Cot': 'sum',
        'Monto Diferencia Prima Pura': ['sum', 'count'],
        'Monto Diferencia Prima Tarifa': ['sum', 'count']
    }).reset_index()

    df_grouped.columns = ['PRODUCTO', 'CANTIDAD', 'PRIMA TÉCNICA ART', 'MIN PRIMA PURA COT', 'MIN PRIMA TARIFA COT', 'MAX PRIMA PURA COT', 'Total Monto Diferencia Prima Pura', 'Count Prima Pura', 'Total Monto Diferencia Prima Tarifa', 'Count Prima Tarifa']
    return df_grouped

In [ ]:
df_new = process_resumen(df)
df_new.to_excel('resumen_auto.xlsx', index=False)